# Clase que implementa un Algoritmo evolutivo

Cada individuo se forma por un cromosoma formado por los siguientes genes:

**Hiperparámetros estructurales**:

- *loopback_window*: 
    rango:  0,7 
    bits:   3
    coding: 2^i
    values: 0=1, 1=2, 2=4, 3=8, 4=16, 5=32, 6=64, 7=128
- *forward_window*:
    rango:  0,7
    bits:   3
    coding: 1+i
    values: 0=1, 1=2, 2=3, 3=4, 4=5, 5=6, 6=7, 7=8
- *num_lstm_layers*:
    rango:  0,3
    bits:   2
    coding: 1+i
    values: 0=1, 1=2, 2=3, 3=4
- *num_dense_layers*: 
    rango:  0,1
    bits:   1
    coding: 2+i
    values: 0=1, 1=2
- *num_cells_per_layer*:
    rango:  (0,3)
    bits:   4
    coding: default: 32, bit0=64 bit1=128 bit2=256 bit3=512 
    values: 0=32, 1=64, 2=128, 3=192, ..., 15=960=512+256+128+64

**Hiperparámetros de entrenamiento**:

- *batch_size*: 
    rango:  (0,3)
    bits:   4
    coding: default: 1, bit0=8 bit1=16 bit2=32 bit3=64 
    values: 0=1, 1=8, 2=16, 3=24, ..., 15=120=64+32+16+8
- *suffling_enable*: rango (0,1), se codifica con 1 bits: 0=>(False), 1=>(True)
    rango:  (0,1)
    bits:   1
    coding: (i) 
    values: 0=False, 1=True

En resumen, cada individuo consta de 3+3+2+1+4+4+1 = 18 bits

In [25]:
from PredictiveNet import PredictiveNet
from deap import base, creator, tools, algorithms
from scipy.stats import bernoulli
from bitstring import BitArray
import numpy as np
import pickle
import sys
import traceback

#######################################################################
#######################################################################
#######################################################################

class Chromosome:
    def __init__(self, deap_bin_cr, verbose):
        """Crea un cromosoma a partir de los valores de un cromosoma binario deap
           Args:
               deap_bin_cr: Cromosoma binario generado por deap.toolbox
               verbose: Modo de depuración
        """
        self.verbose = verbose
        if self.verbose=='full':
            print('Creando cromosoma...')
            
        self.chromosome =[BitArray(deap_bin_cr[0:3]).uint, BitArray(deap_bin_cr[3:6]).uint, BitArray(deap_bin_cr[6:8]).uint,
                          BitArray(deap_bin_cr[8:9]).uint, BitArray(deap_bin_cr[9:13]).uint, BitArray(deap_bin_cr[13:17]).uint,
                          BitArray(deap_bin_cr[17:18]).uint]
        if self.verbose=='full':
            print('Cromosoma resultante:', self.chromosome)        
        
        self.num_genes = len(self.chromosome)        
        self.genes = self.decode_genes()
        if self.verbose=='full':
            print('Genes del cromosoma:', self.genes)

            
    #-------------------------------------------------------------
    def decode_genes(self):
        """Decodifica un chromosoma en sus genes. En este caso contamos con 7 genes:
           lbw,fww,nll,ndl,nlc,bs,sf
           
           Returns:
               Código genético del cromosoma como list[7]
        """        
        genes = []
        chromo = self.chromosome
        # lbw
        genes.append(int(2**chromo[0]))
        # fww
        genes.append(int(1+chromo[1]))
        # nll
        genes.append(int(1+chromo[2]))
        # ndl
        genes.append(int(1+chromo[3]))
        # nlc
        sum = 0
        if chromo[4]==0:
            sum = 32
        else:
            if (chromo[4] & 1):
                sum+=64
            if (chromo[4] & 2):
                sum+=128
            if (chromo[4] & 4):
                sum+=256
            if (chromo[4] & 8):
                sum+=512
        genes.append(int(sum))
        # bs
        sum=0
        if chromo[5]==0:
            sum = 1
        else:
            if (chromo[5] & 1):
                sum+=8
            if (chromo[5] & 2):
                sum+=16
            if (chromo[5] & 4):
                sum+=32
            if (chromo[5] & 8):
                sum+=64
        genes.append(int(sum))
        # sf
        genes.append(True if chromo[6]==1 else False)
        return genes        

#######################################################################
#######################################################################
#######################################################################

class Individual:
    def __init__(self, deap_bin_cr, verbose):
        """Crea un un individuo con un cromosoma binario deap
           Args:
               deap_bin_cr: Cromosoma binario generado por deap.toolbox
               verbose: Modo de depuración
        """
        self.verbose = verbose
        if self.verbose=='full':
            print('Creando individuo a partir de cromosoma:', deap_bin_cr)
        
            
        self.chromosome = Chromosome(deap_bin_cr, verbose)
        self.genes = self.chromosome.genes 
        self.name = str(self.genes)
        if self.verbose=='full':
            print('Nombre del individuo:', self.name)
        
        self.nn = PredictiveNet(  self.name,
                                  loopback_window=self.genes[0], 
                                  forward_window=self.genes[1], 
                                  num_lstm_layers=self.genes[2], 
                                  num_dense_layers=self.genes[3],
                                  num_cells=self.genes[4], 
                                  batch_size=self.genes[5],
                                  suffle_enable= self.genes[6],
                                  tvt_csv_file='EURUSD_H1.csv',
                                  verbose=self.verbose)        
        if self.verbose=='full':
            print('Individuo creado nn={}'.format(self.nn))
        
                
    #-------------------------------------------------------------
    def summary(self):
        """ Visualiza un report del individuo 
        """
        print('Ind Name: {}, Chromosome: {}, Genes: {}, Model: {}'.format(self.name, self.chromosome, self.genes, self.nn.model.summary()))

        
#######################################################################
#######################################################################
#######################################################################

class GA:
    def __init__(self, **kwargs):
        """Constructor del Algoritmo Genético
           Args:
               popsize: Tamaño de la población (0)
               chromosome_len: Tamaño del cromosoma de los individuos (0)
               numgen: Número de generaciones (0)
               pcross: Probabilidad de cruce (0.0)
               pmut: Probabilidad de mutación (0.0)
               psel: Probabilidad de selección (0.)
               verbose: Modo de depuración ('disabled')               
        """      
        self.toolbox = None
        self.stats = None
        self.hallOfFame = None
        self.verbose='disabled'
        self.chromosome_len = 0
        self.popsize = 0
        self.numgen = 0
        self.pcross = 0.0
        self.pmut = 0.0
        self.psel = 0.0
        _build_pop = False
        for key,value in kwargs.items():        
            if key=='popsize': 
                self.popsize = value
            elif key=='chromosome_len':
                self.chromosome_len = value
            elif key=='numgen': 
                self.numgen = value
            elif key=='pcross': 
                self.pcross = value
            elif key=='pmut': 
                self.pmut = value
            elif key=='psel': 
                self.psel = value
            elif key=='verbose':
                self.verbose = value
       
        # As we are trying to minimize the RMSE score, that's why using -1.0. 
        # In case, when you want to maximize accuracy for instance, use 1.0
        print('Creando GA: chromo_len={}, popsize={}'.format(self.chromosome_len, self.popsize))
        creator.create('FitnessMax', base.Fitness, weights = (1.0,))
        creator.create('Individual', list , fitness = creator.FitnessMax)
        toolbox = base.Toolbox()
        toolbox.register('binary', bernoulli.rvs, 0.5)
        toolbox.register('individual', tools.initRepeat, creator.Individual, toolbox.binary, n = self.chromosome_len)
        toolbox.register('population', tools.initRepeat, list , toolbox.individual)
        toolbox.register('mate', tools.cxOrdered)
        toolbox.register('mutate', tools.mutShuffleIndexes, indpb = 0.6)
        toolbox.register('select', tools.selRoulette)
        toolbox.register('evaluate', self.evaluate_individual)
        self.population = toolbox.population(n = self.popsize)
        self.toolbox = toolbox        
        self.summary()
                                 
    #-------------------------------------------------------------
    def summary(self):
        print('Verbose mode:', self.verbose)
        print('Population: ', self.population)
                                 
    #-------------------------------------------------------------
    def set_verbose_mode(self, mode):
        self.verbose = mode
        print('Set verbose mode to:', self.verbose)
                                 
    #-------------------------------------------------------------
    def evaluate_individual(self, chromosome):
        if self.verbose=='full':
            print('Evaluando individuo con cromosoma:', chromosome)
        ind = Individual(chromosome, self.verbose)
        
        if self.verbose=='full' or self.verbose=='result':
            print('Iniciando entrenamiento de individuo con genes:', ind.genes)
        history = ind.nn.train_validate(2)
        
        if self.verbose=='full':
            print('Iniciando evaluación...')
        loss,acc = ind.nn.test_eval()
        
        if self.verbose=='full' or self.verbose=='result':
            print('Resultado loss={}, acc={}'.format(loss,acc))      
        
        # en caso de que acc = 0 el resultado será -loss
        if acc == 0:
            acc = -loss
        return [acc]
                        
    #-------------------------------------------------------------
    def execute(self, numgens=None):
        generations = self.numgen
        verbose = False
        if numgens is not None:
            generations = numgens
        if self.verbose=='full' or self.verbose=='result':
            _verbose = True
            print('Iniciando GA en {} generaciones: pmut={}, pcross={}'.format(generations, self.pmut, self.pcross))
        self.stats = tools.Statistics()
        self.stats.register("mean", np.mean)
        self.stats.register("max", max)
        self.hallOfFame = tools.HallOfFame(min(3,self.popsize))
        r = algorithms.eaSimple(self.population, self.toolbox, cxpb = self.pcross, mutpb = self.pmut, ngen = generations, verbose = _verbose)
        return r
        
#######################################################################
#######################################################################
#######################################################################

#-------------------------------------------------------------
def save_to_file(obj, filepath):
    with open(filepath, 'wb') as f:
        try:
            pickle.dump(obj, f)
        except MemoryError as error:
            exc_type, exc_value, exc_traceback = sys.exc_info()
            print('type:', exc_type, 'value:', exc_value)
        
        
#-------------------------------------------------------------
def load_from_file(filepath):
    with open(filepath, 'rb') as f:
        obj = pickle.load(f)        
    return obj    


In [27]:
ga = GA(popsize=10, chromosome_len=18, numgen=50, pcross=0.4, pmut=0.1, psel=0.5, verbose='result')

Creando GA: chromo_len=18, popsize=2
Verbose mode: result
Population:  [[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0], [1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1]]


In [21]:
ga.execute()

Iniciando GA en 50 generaciones: pmut=0.1, pcross=0.4
file "[4, 1, 3, 1, 512, 120, True]_fitlog.csv" updated
Iniciando entrenamiento de individuo con genes: [4, 1, 3, 1, 512, 120, True]
Train on 71040 samples, validate on 17760 samples
Epoch 1/2
 - 172s - loss: 0.0082 - acc: 2.8153e-05 - val_loss: 0.0528 - val_acc: 0.0000e+00
Epoch 2/2
 - 192s - loss: 0.0259 - acc: 2.8153e-05 - val_loss: 0.1919 - val_acc: 0.0000e+00
Resultado loss=0.1100196032101763, acc=0.0


TypeError: Both weights and assigned values must be a sequence of numbers when assigning to values of <class 'deap.creator.FitnessMax'>. Currently assigning value(s) 0.0 of <class 'numpy.float64'> to a fitness with weights (1.0,).

In [28]:
logbook = ga.execute(1)

Iniciando GA en 1 generaciones: pmut=0.1, pcross=0.4
file "[128, 5, 1, 1, 64, 96, False]_fitlog.csv" updated
Iniciando entrenamiento de individuo con genes: [128, 5, 1, 1, 64, 96, False]
Train on 70656 samples, validate on 17664 samples
Epoch 1/2
 - 81s - loss: 0.0054 - acc: 0.2002 - val_loss: 0.0059 - val_acc: 0.2579
Epoch 2/2
 - 33s - loss: 0.0018 - acc: 0.1998 - val_loss: 0.0036 - val_acc: 0.2719
Resultado loss=0.00039156331886507404, acc=0.2224237865581169
file "[64, 6, 1, 2, 768, 80, True]_fitlog.csv" updated
Iniciando entrenamiento de individuo con genes: [64, 6, 1, 2, 768, 80, True]
Train on 70720 samples, validate on 17680 samples
Epoch 1/2
 - 989s - loss: 0.0047 - acc: 0.1702 - val_loss: 0.0703 - val_acc: 0.1796
Epoch 2/2
 - 995s - loss: 0.0028 - acc: 0.1610 - val_loss: 0.0625 - val_acc: 0.1248
Resultado loss=0.0061504795995337485, acc=0.16418703506907545
gen	nevals
0  	2     
file "[128, 5, 1, 1, 64, 96, False]_fitlog.csv" updated
Iniciando entrenamiento de individuo con gene

In [29]:
logbook

([[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0],
  [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0]],
 [{'gen': 0, 'nevals': 2}, {'gen': 1, 'nevals': 2}])